In [1]:
# 多分类问题 https://www.kaggle.com/c/otto-group-product-classification-challenge/data?select=train.csv
import pandas as pd
import numpy as np
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression